

```
# This is formatted as code
```

# DMA Fall 20

**Note** : This entire lab will be manually evaluated.

Name : 'Yin Qiu'
Collaborator : ''

# Lab 4: Neural Networks #


In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
!wget http://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv
!wget http://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2020-09-21 01:51:59--  http://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv
Resolving people.ischool.berkeley.edu (people.ischool.berkeley.edu)... 128.32.78.16
Connecting to people.ischool.berkeley.edu (people.ischool.berkeley.edu)|128.32.78.16|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv [following]
--2020-09-21 01:51:59--  https://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv
Connecting to people.ischool.berkeley.edu (people.ischool.berkeley.edu)|128.32.78.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105581 (103K) [text/csv]
Saving to: ‘lab_4_training.csv.1’

lab_4_training.csv. 100%[===================>] 103.11K   252KB/s    in 0.4s    

2020-09-21 01:52:00 (252 KB/s) - ‘lab_4_training.csv.1’ saved [105581/105581]

URL transformed to HTTPS due to an HSTS policy
--2020-09-21 01:52:00--  http

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,1303,male,20,second,green,73.0,210.0,0,1,10.0,Yes,5.0,50.0,1.0,15.0
1,36,male,20,third,other,71.0,90.0,1,0,15.0,Yes,4.0,10.0,0.0,1.0
2,489,male,22,fourth,hazel,75.0,200.0,0,1,1.0,Yes,2.0,150.0,1.0,10.0
3,1415,male,19,second,brown,72.0,35.0,2,2,20.0,Yes,5.0,100.0,0.0,7.0
4,616,male,22,fourth,hazel,71.0,15.0,2,1,10.0,Yes,7.0,10.0,0.0,5.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of 'gender'. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives)

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [11]:
df1 = df_train.gender.value_counts()
df1

female    855
male      735
Name: gender, dtype: int64

In [33]:
# majority class is female; calculate accuracy if all predictions are female
accuracy_q1a = df1['female']/df1.sum()
accuracy_q1a

0.5377358490566038

###ANSWER: 
Accuracy is 0.5377358490566038

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [13]:
df2 = df_test.gender.value_counts()
df2

female    208
male      190
Name: gender, dtype: int64

In [34]:
accuracy_q1b = df2['female']/df2.sum()
accuracy_q1b

0.5226130653266332

###ANSWER: 
Accuracy is 0.5226130653266332

***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation (eg: scikit-learn) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (sigmoid/logistc) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (lbfgs). It is possible to specify logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the Height feature of the dataset to predict the Gender. You will have to change Gender to a 0 and 1 class. After training, use your trained model to predict the class using the height feature from the training set. What was the accuracy of this prediction?

In [27]:
X_train = df_train[['height']]
y_train = df_train['gender'].replace({'female': 0, 'male': 1}, inplace=False)

clf = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=10, random_state=42, verbose=True)

clf.fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
print('Accuracy on training: ', accuracy_score(y_train, y_train_pred))

Accuracy on training:  0.8465408805031447


###ANSWER: 
The accuracy of the prediction on training data is 0.8465408805031447

**Question 2.b**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the Height feature values from the test set. What is the accuracy of this model on the test set?

In [28]:
X_test = df_test[['height']]
y_test = df_test['gender'].replace({'female': 0, 'male': 1}, inplace=False)

y_test_pred = clf.predict(X_test)
print('Accuracy on testing: ', accuracy_score(y_test, y_test_pred))

Accuracy on testing:  0.8542713567839196


###ANSWER: 
The accuracy of the prediction on testing data is 0.8542713567839196

**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the height feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature

In [32]:
# Use Standard Scaler to normalize X train and X test
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

clf_2c = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=10, random_state=42, verbose=True)

clf_2c.fit(X_train_scaled, y_train)
y_train_pred_2c = clf_2c.predict(X_train_scaled)
print('Accuracy on standardized training set: ', accuracy_score(y_train, y_train_pred_2c))

y_test_pred_2c = clf_2c.predict(X_test_scaled)
print('Accuracy on standardized testing set: ', accuracy_score(y_test, y_test_pred_2c))

Accuracy on standardized training set:  0.8465408805031447
Accuracy on standardized testing set:  0.8542713567839196


###ANSWER: 
Using standardization scaler, the accuracy on training is 0.8465408805031447, and the accuracy on testing is 0.8542713567839196

***

### Question 3###
The rest of features in this dataset barring a few are categorical. No ML method accepts categorical features, so transform year, eyecolor, exercise into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set. What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50.

In [39]:
df_train['year'].replace('first"','first', inplace=True) # fix a typo in the dataset
df_test['year'].replace('first"','first', inplace=True) # fix a typo in the dataset
df3_train = pd.get_dummies(data=df_train[['year', 'eyecolor', 'exercise']]) # one-hot encoding for training categorical variables
df3_test = pd.get_dummies(data=df_test[['year', 'eyecolor', 'exercise']]) # one-hot encoding for testing categorical variables

clf_3 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)

clf_3.fit(df3_train, y_train)
y_train_pred3 = clf_3.predict(df3_train)
print('Accuracy of training data of year, eyecolor and exercise: ', accuracy_score(y_train, y_train_pred3))

y_test_pred3 = clf_3.predict(df3_test)
print('Accuracy of testing data of year, eyecolor and exercise: ', accuracy_score(y_test, y_test_pred3))

Accuracy of training data of year, eyecolor and exercise:  0.5779874213836478
Accuracy of testing data of year, eyecolor and exercise:  0.5728643216080402


###ANSWER: 

Accuracy of training data of year, eyecolor and exercise:  0.5779874213836478

Accuracy of testing data of year, eyecolor and exercise:  0.5728643216080402



***
### Question 4###
Using a NN, report the accuracy on  the test set of a model that trained only on the height and the eye color features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original height values (no pre-processing) and eye color as a one-hot?

In [95]:
# no preprocessing height and one-hot encoding eyecolor
df4_train = pd.concat([df_train['height'], pd.get_dummies(data=df_train['eyecolor'])], axis=1)
df4_test = pd.concat([df_test['height'], pd.get_dummies(data=df_test['eyecolor'])], axis=1)

clf_4a = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)
clf_4a.fit(df4_train, y_train)
y_train_pred4a = clf_4a.predict(df4_train)
print('Accuracy of training data of height and eyecolor: ', accuracy_score(y_train, y_train_pred4a))

y_test_pred4a = clf_4a.predict(df4_test)
print('Accuracy of testing data of height and eyecolor: ', accuracy_score(y_test, y_test_pred4a))

Accuracy of training data of height and eyecolor:  0.8446540880503145
Accuracy of testing data of height and eyecolor:  0.8567839195979899


###ANSWER: 
Accuracy of testing data of year and eyecolor is 0.8567839195979899

**Question 4.b**  
What is the accuracy on the test set using the log of height values (applied to both training and testing sets) and eye color as a one-hot?

In [96]:
# log transformation of height and one-hot encoding eyecolor
df4_train['height'] = np.log(np.array(df_train[['height']]))
df4_test['height'] = np.log(np.array(df_test[['height']]))

clf_4b = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)
clf_4b.fit(df4_train, y_train)
y_train_pred4b = clf_4b.predict(df4_train)
print('Accuracy of training data of log(height) and eyecolor: ', accuracy_score(y_train, y_train_pred4b))

y_test_pred4b = clf_4b.predict(df4_test)
print('Accuracy of testing data of log(height) and eyecolor: ', accuracy_score(y_test, y_test_pred4b))

Accuracy of training data of log(height) and eyecolor:  0.8503144654088051
Accuracy of testing data of log(height) and eyecolor:  0.8592964824120602


###ANSWER: 
Accuracy of testing data of log(height) and eyecolor is 0.8592964824120602

**Question 4.c**  
What is the accuracy on the test set using the Z-score of height values and eye color as a one-hot? 

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [97]:
# z-score transformation of height and one-hot encoding eyecolor
from scipy.stats import zscore
df4_train['height'] = zscore(df_train[['height']])
df4_test['height'] = zscore(df_test[['height']])

clf_4c = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)
clf_4c.fit(df4_train, y_train)
y_train_pred4c = clf_4c.predict(df4_train)
print('Accuracy of training data of z-score(height) and eyecolor: ', accuracy_score(y_train, y_train_pred4c))

y_test_pred4c = clf_4c.predict(df4_test)
print('Accuracy of testing data of z-score(height) and eyecolor: ', accuracy_score(y_test, y_test_pred4c))

Accuracy of training data of z-score(height) and eyecolor:  0.8540880503144654
Accuracy of testing data of z-score(height) and eyecolor:  0.8693467336683417


###ANSWER: 
Accuracy of testing data of z-score(height) and eyecolor is  0.8693467336683417

***
### Question 5 ###
Repeat question 4 for exercise hours + eye color

In [110]:
# no preprocessing exercise hours and one-hot encoding eyecolor
df5_train = pd.concat([df_train['exercisehours'], pd.get_dummies(data=df_train['eyecolor'])], axis=1)
df5_test = pd.concat([df_test['exercisehours'], pd.get_dummies(data=df_test['eyecolor'])], axis=1)

clf_5a = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)
clf_5a.fit(df5_train, y_train)
y_train_pred5a = clf_5a.predict(df5_train)
print('Accuracy of training data of exercise hours and eyecolor: ', accuracy_score(y_train, y_train_pred5a))

y_test_pred5a = clf_5a.predict(df5_test)
print('Accuracy of testing data of exercise hours and eyecolor: ', accuracy_score(y_test, y_test_pred5a))

Accuracy of training data of exercise hours and eyecolor:  0.5880503144654088
Accuracy of testing data of exercise hours and eyecolor:  0.5778894472361809


In [113]:
# log transformation of exercise hours and one-hot encoding eyecolor
exer_train = np.array(df_train[['exercisehours']])
df5_train['exercisehours'] = np.log(np.where(exer_train==0, 0.00001, exer_train)) # replace 0 to 0.00001 for log transformation

exer_test = np.array(df_test[['exercisehours']])
df5_test['exercisehours'] = np.log(np.where(exer_test==0, 0.00001, exer_test)) # replace 0 to 0.00001 for log transformation

clf_5b = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)

clf_5b.fit(df5_train, y_train)
y_train_pred5b = clf_5b.predict(df5_train)
print('Accuracy of training data of log(exercise hours) and eyecolor: ', accuracy_score(y_train, y_train_pred5b))

y_test_pred5b = clf_5b.predict(df5_test)
print('Accuracy of testing data of log(exercise hours) and eyecolor: ', accuracy_score(y_test, y_test_pred5b))

Accuracy of training data of log(exercise hours) and eyecolor:  0.5861635220125786
Accuracy of testing data of log(exercise hours) and eyecolor:  0.5753768844221105


In [115]:
# z-score transformation of exercise hours and one-hot encoding eyecolor
df5_train['exercisehours'] = zscore(df_train[['exercisehours']])
df5_test['exercisehours'] = zscore(df_test[['exercisehours']])

clf_5c = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)
clf_5c.fit(df5_train, y_train)
y_train_pred5c = clf_5c.predict(df5_train)
print('Accuracy of training data of z-score(exercisehours) and eyecolor: ', accuracy_score(y_train, y_train_pred5c))

y_test_pred5c = clf_5c.predict(df5_test)
print('Accuracy of testing data of z-score(exercisehours) and eyecolor: ', accuracy_score(y_test, y_test_pred5c))

Accuracy of training data of z-score(exercisehours) and eyecolor:  0.5937106918238994
Accuracy of testing data of z-score(exercisehours) and eyecolor:  0.5753768844221105


###ANSWER: 
Accuracy of testing data of exercise hours and eyecolor is 0.5778894472361809

Accuracy of testing data of log(exercise hours) and eyecolor is 0.5753768844221105

Accuracy of testing data of z-score(exercisehours) and eyecolor is 0.5753768844221105

***
### Question 6###
Combine the features from question 3, 4, and 5 (year, eyecolor, exercise, height, exercise hours). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [117]:
# Accroding to q4 and q5, z-score(height) and non-preprocessing(exercisehours) are the best normalization methods
df6_train = pd.concat([df3_train, df4_train[['height']], df_train[['exercisehours']]], axis=1)
df6_test = pd.concat([df3_test, df4_test[['height']], df_test[['exercisehours']]], axis=1)

clf_6 = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', 
                    max_iter=100, n_iter_no_change=50, random_state=42, verbose=True)
clf_6.fit(df6_train, y_train)
y_train_pred6 = clf_6.predict(df6_train)
print('Accuracy of training data of z-score(exercisehours) and eyecolor: ', accuracy_score(y_train, y_train_pred6))

y_test_pred6 = clf_6.predict(df6_test)
print('Accuracy of testing data of z-score(exercisehours) and eyecolor: ', accuracy_score(y_test, y_test_pred6))

Accuracy of training data of z-score(exercisehours) and eyecolor:  0.8572327044025158
Accuracy of testing data of z-score(exercisehours) and eyecolor:  0.864321608040201


###ANSWER: 
Accuracy of testing data of z-score(exercisehours) and eyecolor is 0.864321608040201

***
### Question 7- Bonus (10%)###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. 

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.


In [ ]:
# YOUR CODE HERE

###ANSWER: 